In [2]:
import fastf1
import fastf1.plotting
import pandas as pd 
import numpy as np
import re
from matplotlib import pyplot as plt
print(fastf1.__version__)
from fastf1 import utils
from fastf1.core import Laps
import fastf1.plotting
import matplotlib.pyplot as plt


fastf1.plotting.setup_mpl(misc_mpl_mods=False)

3.0.7


In [ ]:
# QUALY POSITIONS

fastf1.Cache.enable_cache("/Users/ainefernandez/documents/itam/8SEMESTRE/F1 Python")

year = 2024
ses = 'Q'

# Initialize an empty list to store position data
position_data = []

# Loop over all Grand Prix (assuming there are 24 races in the season)
for i in range(1, 25):
    session = fastf1.get_session(year, i, ses)
    session.load()
    
    # Extract unique drivers' abbreviations from the results
    drivers = pd.unique(session.results['Abbreviation'])
    
    for driver in drivers:
        results = session.get_driver(driver)
        qualy = results['Position']
        
        # Determine the qualifying segment
        if qualy <= 10:
            q = "Q3"
        elif 10 < qualy <= 15:
            q = "Q2"
        else:
            q = "Q1"
        
        # Append the data to the list
        position_data.append({'Driver': driver, 'Position': qualy, 'Q': q, 'Qualy': 1})

# Create a DataFrame from the collected data
position = pd.DataFrame(position_data)

print(position)



In [ ]:
position.to_csv("FSQUALYtype.csv", index=False)

In [ ]:
# Full Season QUALY
import fastf1
import pandas as pd

fastf1.Cache.enable_cache("/Users/ainefernandez/documents/itam/8SEMESTRE/F1 Python")

year = 2024
ses = 'Q'
sprints=["China","Miami", "Austria", "Austin", "Brazil","Qatar"]  

# Initialize an empty list to store position data
position_data = []

# Loop over all Grand Prix (assuming there are 24 races in the season)
for i in range(1, 25):
    session = fastf1.get_session(year, i, ses)
    session.load()
    
    # Extract unique drivers' abbreviations from the results
    drivers = pd.unique(session.results['Abbreviation'])
    
    for driver in drivers:
        results = session.get_driver(driver)
        qualy = results['Position']
        
        # Append the data to the list
        position_data.append({'Driver': driver, 'Position': qualy})


        
        
# Create a DataFrame from the collected data
position = pd.DataFrame(position_data)

print(position)

# Group by Driver and calculate the mean qualifying position
grouped_position_mean = position.groupby('Driver')['Position'].mean()
print(grouped_position_mean)

# Save the grouped mean positions to a CSV file
grouped_position_mean.to_csv("FSAVGQUALY.csv", index=True)




In [ ]:
import fastf1
import pandas as pd

# Enable caching for FastF1
fastf1.Cache.enable_cache("/Users/ainefernandez/documents/itam/8SEMESTRE/F1 Python")

# Parameters
year = 2024
ses = 'R'
drivers = ["VER", "PER", "LEC", "SAI", "RUS", "HAM", "NOR", "PIA", "OCO", "GAS"]

# Initialize an empty list to store podium data
podium_data = []

# Dictionary to map positions to podium places
podiums = {1: "First", 2: "Second", 3: "Third"}

# Loop over all races in the season
for i in range(1, 25):
    # Load the race session
    session = fastf1.get_session(year, i, ses)
    session.load()
    
    # Filter session results for the specified drivers
    for _, row in session.results.iterrows():
        if row['Abbreviation'] in drivers and row['Position'] in [1, 2, 3]:
            podium_data.append({
                'Driver': row['Abbreviation'],
                'Position': podiums[row['Position']],
                'Podiums': 1
            })

# Convert collected data to a DataFrame
position = pd.DataFrame(podium_data)

# Save the results to a CSV file
position.to_csv("PodiumsFS.csv", index=False)

print(position)



In [ ]:
# Avg Gap to Pole
import fastf1
import pandas as pd
from fastf1.core import Laps

fastf1.Cache.enable_cache("/Users/ainefernandez/documents/itam/8SEMESTRE/F1 Python")

year = 2024
ses = 'Q'

# Initialize a list to store gap data
gap_data = []

# Loop over all completed Grand Prix sessions (1–14 in this case)
for i in range(1, 25):
    session = fastf1.get_session(year, i, ses)
    session.load()
    
    # Get unique drivers
    drivers = pd.unique(session.laps['Driver'])
    fastest_laps_list = []
    
    # Collect the fastest lap for each driver
    for drv in drivers:
        try:
            driver_fastest_lap = session.laps.pick_driver(drv).pick_fastest()
            fastest_laps_list.append(driver_fastest_lap)
        except ValueError:
            # Handle cases where a driver doesn't have a valid lap time
            continue
    
    # Convert the list of fastest laps into a Laps object and sort by lap time
    fastest_laps = Laps(fastest_laps_list).sort_values(by='LapTime').reset_index(drop=True)
    
    # Get the pole position lap (fastest lap overall)
    if not fastest_laps.empty:
        pole_lap = fastest_laps.iloc[0]
        
        # Calculate gaps to the pole lap and store the data
        for _, row in fastest_laps.iterrows():
            gap = (row['LapTime'] - pole_lap['LapTime']).total_seconds()  # Gap in seconds
            gap_data.append({'Driver': row['Driver'], 'Gap': gap})

# Create a DataFrame from the collected gap data
position = pd.DataFrame(gap_data)

# Print the gap data
print(position)

# Group by driver and calculate the average gap
avg_gap_to_pole = position.groupby('Driver')['Gap'].mean()
print(avg_gap_to_pole)

# Save the average gap data to a CSV file
# avg_gap_to_pole.to_csv("AvgGapToPole.csv", index=True)


In [ ]:
import fastf1
import pandas as pd

# Enable FastF1 cache
fastf1.Cache.enable_cache("/Users/ainefernandez/documents/itam/8SEMESTRE/F1 Python")

year = 2024
ses = 'Q'

# Initialize an empty list to store position data
position_data = []

# Loop over all Grand Prix (assuming there are 24 races in the season)
for i in range(1, 25):
    session = fastf1.get_session(year, i, ses)
    session.load()
    
    # Extract unique drivers' abbreviations from the results
    drivers = pd.unique(session.results['Abbreviation'])
    
    for driver in drivers:
        # Get the driver's position
        qualy_position = session.results.loc[session.results['Abbreviation'] == driver, 'Position'].values[0]
        
        # Determine the qualifying segment
        if qualy_position <= 10:
            q = "Q3"
        elif 10 < qualy_position <= 15:
            q = "Q2"
        else:
            q = "Q1"
        
        # Assign points based on the qualifying position
        if qualy_position == 1:
            points = 25
        elif qualy_position == 2:
            points = 18
        elif qualy_position == 3:
            points = 15
        elif qualy_position == 4:
            points = 12
        elif qualy_position == 5:
            points = 10
        elif qualy_position == 6:
            points = 8
        elif qualy_position == 7:
            points = 6
        elif qualy_position == 8:
            points = 4
        elif qualy_position == 9:
            points = 2
        elif qualy_position == 10:
            points = 1
        else:
            points = 0
        
        # Append the data to the list
        position_data.append({'Driver': driver, 'Position': qualy_position, 'Qualy_Segment': q, 'Points': points})

# Create a DataFrame from the collected data
position = pd.DataFrame(position_data)

# Calculate total points per driver
total_points = position.groupby('Driver', as_index=False)['Points'].sum()

# Rename the column for clarity
total_points.rename(columns={'Points': 'Total_Points'}, inplace=True)

# Display the results
print("Qualifying Results with Points:")
print(position)
print("\nTotal Points by Driver:")
print(total_points)

# Optionally save the results to CSV files
#position.to_csv("QualyPoints2024.csv", index=False)
total_points.to_csv("TotalQualyPoints2024.csv", index=False)


In [ ]:
import fastf1
import pandas as pd

# Enable FastF1 cache
fastf1.Cache.enable_cache("/Users/ainefernandez/documents/itam/8SEMESTRE/F1 Python")

year = 2024
ses = 'R'
total_races = 25  # Update this if the total number of races is different

# Initialize the DataFrame
position = pd.DataFrame(columns=['Driver', 'Points_count', 'Total_races', 'Points_percentage'])

for i in range(1, total_races):  # Loop through all races
    session = fastf1.get_session(year, i, ses)
    session.load()
    
    # Get unique drivers and their results
    results = session.results
    drivers = pd.unique(results['Abbreviation'])
    
    for driver in drivers:
        # Get the driver's position
        qualy_position = results.loc[results['Abbreviation'] == driver, 'Position'].values[0]
        
        # Determine if they scored points (Q3 finish)
        q3_count = 1 if qualy_position <= 10 else 0
        
        # Update or add driver information
        if driver in position['Driver'].values:
            # Update existing driver record
            position.loc[position['Driver'] == driver, 'Points_count'] += q3_count
            position.loc[position['Driver'] == driver, 'Total_races'] += 1
        else:
            # Add a new driver record
            new_entry = pd.DataFrame({'Driver': [driver], 'Points_count': [q3_count], 'Total_races': [1], 'Points_percentage': [0]})
            position = pd.concat([position, new_entry], ignore_index=True)

# Calculate the Q3 percentage
position['Points_percentage'] = (position['Points_count'] / position['Total_races']) * 100

# Output the results
print(position)

# Save to CSV
position.to_csv("FSPointsc.csv", index=False)


In [ ]:

import fastf1
import pandas as pd

# Enable caching for FastF1
fastf1.Cache.enable_cache("/Users/ainefernandez/documents/itam/8SEMESTRE/F1 Python")

# Parameters
year = 2024
ses = 'R'  # 'R' for Race session
sprints = [5, 6, 11, 19, 21, 23]  # Races with sprints

# Initialize an empty list to store position data
position_data = []

# Loop over all Grand Prix (assuming there are 24 races in the season)
for i in range(1, 25):
    # Load the main race session
    session = fastf1.get_session(year, i, ses)
    session.load()
    
    # Extract driver results and points
    for _, row in session.results.iterrows():
        position_data.append({'Driver': row['Abbreviation'], 'Points': row['Points']})
    
    # Handle sprint sessions if applicable
    if i in sprints: 
        sprint_session = fastf1.get_session(year, i, "Sprint")
        sprint_session.load()
        
        # Extract sprint driver results and points
        for _, row in sprint_session.results.iterrows():
            position_data.append({'Driver': row['Abbreviation'], 'Points': row['Points']})

# Convert collected data to a DataFrame
position = pd.DataFrame(position_data)

# Calculate total and average points per driver
pointstotal = position.groupby('Driver')['Points'].sum()
avgpoints = position.groupby('Driver')['Points'].mean()

# Display the results
print("Position Data:")
print(position)

print("\nTotal Points per Driver:")
print(pointstotal)

print("\nAverage Points per Driver:")
print(avgpoints)






In [ ]:
pointstotal.to_csv('pointstotal.csv', index=False)

In [ ]:
import fastf1
import pandas as pd

# Enable caching for FastF1
fastf1.Cache.enable_cache("/Users/ainefernandez/documents/itam/8SEMESTRE/F1 Python")

# Parameters
year = 2024
ses = 'R'

# Initialize an empty list to collect race data
race_data = []

# Loop over the first 14 races of the season
for i in range(1, 25):
    # Load the race session
    session = fastf1.get_session(year, i, ses)
    session.load()
    
    # Extract results for all drivers
    for _, row in session.results.iterrows():
        driver = row['Abbreviation']
        qualy = row['Position']
        status = row['Status']
        
        # Determine finish type based on position and status
        if status in ['Finished', '+1 Lap', '+2 Laps', '+3 Laps', '+4 Laps', '+5 Laps', '+6 Laps', '+7 Laps']:
            if qualy == 1:
                finish = "Win"
            elif qualy in [2, 3]:
                finish = "Podium"
            elif qualy <= 10:
                finish = "Points"
            else:
                finish = "Out of the points"
        else:
            finish = "DNF"
        
        # Append the data to the list
        race_data.append({
            'Driver': driver,
            'Position': qualy,
            'Status': status,
            'Finish': finish,
            'Race': i
        })

# Convert collected data to a DataFrame
position = pd.DataFrame(race_data)

# Save the DataFrame to a CSV file
position.to_csv("FSRaceType.csv", index=False)

print(position)


In [ ]:
import fastf1
import pandas as pd

# Enable cache
fastf1.Cache.enable_cache("/Users/ainefernandez/documents/itam/8SEMESTRE/F1 Python")

# Set parameters
year = 2024
ses = 'R'

# Create an empty DataFrame to store results
position = pd.DataFrame(columns=['Driver', 'GridPosition', 'Position', 'Diff', 'Status'])

# Loop through races
for i in range(1, 25):  # 24 races in the season
    try:
        # Load the session data
        session = fastf1.get_session(year, i, ses)
        session.load()
        
        # Get driver data
        drivers = pd.unique(session.results['Abbreviation'])
        
        # Process each driver
        for driver in drivers:
            # Extract driver results
            driver_results = session.results[session.results['Abbreviation'] == driver]
            
            # Get race position, grid position, and status
            race = driver_results.iloc[0]['Position']
            grid = driver_results.iloc[0]['GridPosition']
            status = driver_results.iloc[0]['Status']
            
            # Calculate the difference
            diff = grid - race
            
            # Create a temporary DataFrame
            temp_df = pd.DataFrame({
                'Driver': [driver],
                'GridPosition': [grid],
                'Position': [race],
                'Diff': [diff],
                'Status': [status]
            })
            
            # Concatenate to the main DataFrame
            position = pd.concat([position, temp_df], ignore_index=True)
    except Exception as e:
        print(f"Error processing race {i}: {e}")

# Filter for races with status 'Finished' or similar
finished_positions = position[position['Status'].isin(['Finished', '+1 Lap', '+2 Laps'])]

# Calculate the total positions gained or lost by each driver
total_diff_by_driver = finished_positions.groupby('Driver')['Diff'].sum().reset_index()

# Rename the column for clarity
total_diff_by_driver.rename(columns={'Diff': 'TotalDiff'}, inplace=True)

# Print the result
print(total_diff_by_driver)

# Save to CSV if needed
total_diff_by_driver.to_csv("FSPositionsgained.csv", index=False)


In [ ]:
import fastf1
import pandas as pd

# Enable cache
fastf1.Cache.enable_cache("/Users/ainefernandez/documents/itam/8SEMESTRE/F1 Python")

# Set parameters
year = 2024
session_type = 'R'  # Race sessions

# Create a DataFrame to store race statistics
race_stats = pd.DataFrame(columns=['GP', 'Driver', 'LapNumber', 'AverageLapTime', 'FastestLapTime', 'PercentageDifference'])

# Iterate through the 25 races of the season
for race_number in range(1, 25):  # From 1 to 25
    try:
        # Load the race session
        session = fastf1.get_session(year, race_number, session_type)
        session.load()

        # Get all valid laps
        laps = session.laps.pick_quicklaps()

        # Calculate the average lap time
        average_lap_time = laps['LapTime'].mean().total_seconds()

        # Find the fastest lap
        fastest_lap = laps.loc[laps['LapTime'].idxmin()]
        fastest_lap_time = fastest_lap['LapTime'].total_seconds()
        fastest_lap_driver = fastest_lap['Driver']
        fastest_lap_number = fastest_lap['LapNumber']

        # Calculate the percentage difference
        percentage_difference = ((average_lap_time - fastest_lap_time) / average_lap_time) * 100

        # Add the results to the DataFrame
        race_stats = pd.concat([race_stats, pd.DataFrame({
            'GP': [session.event['EventName']],
            'Driver': [fastest_lap_driver],
            'LapNumber': [fastest_lap_number],
            'AverageLapTime': [average_lap_time],
            'FastestLapTime': [fastest_lap_time],
            'PercentageDifference': [percentage_difference]
        })], ignore_index=True)

    except Exception as e:
        print(f"Error processing race {race_number}: {e}")

# Display the results
print(race_stats)

# Save results to a CSV file
race_stats.to_csv("FastestLap.csv", index=False)


In [ ]:
import fastf1

# Enable cache
fastf1.Cache.enable_cache("/Users/ainefernandez/documents/itam/8SEMESTRE/F1 Python")

# Set parameters
year = 2024
race_number = 24  # Las Vegas Grand Prix

try:
    # Load the Las Vegas Grand Prix session
    session = fastf1.get_session(year, race_number, 'R')  # 'R' for Race
    session.load()

    # Get and print race results
    results = session.results[['Abbreviation', 'Position', 'Time', 'Status', 'Points']]
    print(results)

except Exception as e:
    print(f"Error loading Las Vegas Grand Prix results: {e}")


   

    


In [ ]:
import fastf1
import pandas as pd

# Enable cache
fastf1.Cache.enable_cache("/Users/ainefernandez/documents/itam/8SEMESTRE/F1 Python")



# Initialize DataFrame for storing pole stats
pole_stats = pd.DataFrame(columns=['GP', 'PoleSitter', 'GapToSecond'])

# Loop through all races
year = 2024
for race_number in range(1, 24):  # For races 1 to 25
    try:
        # Load the qualifying session
        session = fastf1.get_session(year, race_number, 'Q')
        session.load()

        # Initialize gap DataFrame
        gapQ = pd.DataFrame(columns=['Driver', 'Team', 'Gap'])
        drivers = pd.unique(session.laps['Driver'])
        list_fastest_laps = []

        # Calculate fastest laps for each driver
        for drv in drivers:
            drvs_fastest_lap = session.laps.pick_driver(drv).pick_fastest()
            if not drvs_fastest_lap.empty:
                list_fastest_laps.append(drvs_fastest_lap)

        # Create a DataFrame of the fastest laps and sort by lap time
        fastest_laps = fastf1.core.Laps(list_fastest_laps).sort_values(by='LapTime').reset_index(drop=True)
        pole_lap = fastest_laps.iloc[0]  # Get the pole position lap

        # Calculate and store gap to pole in seconds
        for index, row in fastest_laps.iterrows():
            gap = (row['LapTime'] - pole_lap['LapTime']).total_seconds()  # Calculate gap in seconds
            pos = {'Driver': row['Driver'], 'Team': row['Team'], 'Gap': gap}
            gapQ = pd.concat([gapQ, pd.DataFrame([pos])], ignore_index=True)

        # Extract the pole sitter and gap to second
        pole_sitter = pole_lap['Driver']
        gap_to_second = gapQ.iloc[1]['Gap'] if len(gapQ) > 1 else None  # Handle case where there's no second-fastest

        # Add to pole stats
        pole_stats = pd.concat([pole_stats, pd.DataFrame({
            'GP': [session.event['EventName']],
            'PoleSitter': [pole_sitter],
            'GapToSecond': [gap_to_second]
        })], ignore_index=True)

    except Exception as e:
        print(f"Error processing race {race_number}: {e}")

# Display the pole stats DataFrame
print(pole_stats)

# Save results to a CSV file
pole_stats.to_csv("PoleMargins2024.csv", index=False)


In [ ]:
fastest_lap_df = pd.read_csv("FastestLap.csv")

# Display the first few rows of the DataFrame to verify the content
print(fastest_lap_df.head())

In [ ]:
avgtimes=fastest_lap_df[["GP","AverageLapTime"]]
avgtimes


In [ ]:
import fastf1
import pandas as pd

lapdata = []

# Iterar sobre todas las carreras
for i in range(1, 26):  # Corregido: range(1, 26) para incluir las carreras de 1 a 25
    try:
        # Cargar la sesión de carrera
        session = fastf1.get_session(2024, i, 'R')
        session.load()  # Cargar los datos de la sesión
        
        # Obtener los pilotos únicos
        drivers = pd.unique(session.laps['Driver'])
        
        # Iterar sobre cada piloto
        for driver in drivers:
            try:
                # Seleccionar las vueltas más rápidas para cada piloto
                laps = session.laps.pick_driver(driver).pick_quicklaps()

                # Convertir LapTime a segundos
                laps['LapTime'] = laps['LapTime'].dt.total_seconds()

                # Añadir los datos de las vueltas a la lista
                lapdata.append(laps)
            
            except Exception as e:
                print(f"Error procesando las vueltas rápidas para el piloto {driver} en la carrera {i}: {e}")
    
    except Exception as e:
        print(f"Error cargando la sesión para la carrera {i}: {e}")

# Concatenar todos los datos de las vueltas
if lapdata:
    all_laps = pd.concat(lapdata, ignore_index=True)

    # Calcular el tiempo promedio de vuelta para cada piloto
    average_laptimes = all_laps.groupby(['Team', 'Driver'])['LapTime'].mean().reset_index()

    # Mostrar el tiempo promedio de vuelta
    print(average_laptimes)

    # Encontrar el tiempo promedio de vuelta más rápido para cada equipo
    fastest_avg_laptime_per_team = average_laptimes.groupby('Team')['LapTime'].min()

    # Mostrar el tiempo promedio más rápido por equipo
    print(fastest_avg_laptime_per_team)

    # Calcular el ritmo de carrera promedio más rápido
    fastest_avg_race_pace = fastest_avg_laptime_per_team.min()

    # Calcular la diferencia en porcentaje al ritmo de carrera promedio más rápido por equipo
    gap_to_fastest_avg_race_pace = ((fastest_avg_laptime_per_team - fastest_avg_race_pace) / fastest_avg_race_pace) * 100

    # Mostrar la diferencia en porcentaje
    print(gap_to_fastest_avg_race_pace)

    # Guardar los resultados en un archivo CSV
    gap_to_fastest_avg_race_pace.to_csv("GapRacePace2024_Percentage.csv")
else:
    print("No se pudieron cargar datos de las vueltas para las carreras.")


In [ ]:
races_2024 = fastf1.get_event_schedule(2024)
races_2024

In [ ]:
import fastf1
import pandas as pd
from fastf1.core import Laps

# Habilitar caché
fastf1.Cache.enable_cache("/Users/ainefernandez/documents/itam/8SEMESTRE/F1 Python")

year = 2024
ses = 'Q'
position = pd.DataFrame(columns=['Driver', 'Gap'])

for i in range(1, 25):
    try:
        # Cargar sesión
        session = fastf1.get_session(year, i, ses)
        session.load()
        
        # Obtener los pilotos únicos
        drivers = pd.unique(session.laps['Driver'])
        list_fastest_laps = []
        
        for drv in drivers:
            try:
                # Obtener la vuelta más rápida de cada piloto
                drvs_fastest_lap = session.laps.pick_driver(drv).pick_fastest()
                list_fastest_laps.append(drvs_fastest_lap)
            except Exception as e:
                print(f"Error al procesar la vuelta más rápida de {drv}: {e}")
                continue
        
        # Ordenar las vueltas más rápidas por tiempo y obtener la vuelta de pole position
        fastest_laps = Laps(list_fastest_laps).sort_values(by='LapTime').reset_index(drop=True)
        if fastest_laps.empty:
            print(f"No se encontraron vueltas rápidas para el GP {i}")
            continue

        pole_lap = fastest_laps.iloc[0]  # Vuelta más rápida (pole)
        pole_time = pole_lap['LapTime'].total_seconds()  # Tiempo de la pole en segundos
        
        # Calcular las diferencias en porcentaje
        for _, row in fastest_laps.iterrows():
            driver_time = row['LapTime'].total_seconds()
            gap_percentage = ((driver_time - pole_time) / pole_time) * 100
            pos = {'Driver': row['Driver'], 'Gap': gap_percentage}
            position = pd.concat([position, pd.DataFrame([pos])], ignore_index=True)
    
    except Exception as e:
        print(f"Error al procesar el GP {i}: {e}")

# Mostrar resultados
print(position)
position.to_csv("AvgGapPole2024.csv")

In [ ]:
grouped=position.groupby("Driver")
average_gaps = grouped['Gap'].mean()
print(average_gaps)
#average_gaps.to_csv("FSAvgGaptopole.csv")

In [35]:
#Points heatmap drivers
from fastf1.ergast import Ergast
fastf1.Cache.enable_cache("/Users/ainefernandez/documents/itam/8SEMESTRE/F1 Python")



ergast = Ergast()
races = ergast.get_race_schedule(2023)  # Races in year 2022
results = []

# For each race in the season
for rnd, race in races['raceName'].items():

    # Get results. Note that we use the round no. + 1, because the round no.
    # starts from one (1) instead of zero (0)
    temp = ergast.get_race_results(season=2023, round=rnd + 1)
    temp = temp.content[0]

    # If there is a sprint, get the results as well
    sprint = ergast.get_sprint_results(season=2023, round=rnd + 1)
    if sprint.content and sprint.description['round'][0] == rnd + 1:
        temp = pd.merge(temp, sprint.content[0], on='driverCode', how='left')
        # Add sprint points and race points to get the total
        temp['points'] = temp['points_x'] + temp['points_y']
        temp.drop(columns=['points_x', 'points_y'], inplace=True)

    # Add round no. and grand prix name
    temp['round'] = rnd + 1
    temp['race'] = race.removesuffix(' Grand Prix')
    temp = temp[['round', 'race', 'driverCode', 'points']]  # Keep useful cols.
    results.append(temp)

# Append all races into a single dataframe
results = pd.concat(results)
races = results['race'].drop_duplicates()



In [36]:
results = results.pivot(index='driverCode', columns='round', values='points')
# Here we have a 22-by-22 matrix (22 races and 22 drivers, incl. DEV and HUL)

# Rank the drivers by their total points
results['total_points'] = results.sum(axis=1)
results = results.sort_values(by='total_points', ascending=False)
print(results['total_points'])
results.drop(columns='total_points', inplace=True)

# Use race name, instead of round no., as column names
results.columns = races



driverCode
VER    575.0
PER    285.0
HAM    234.0
LEC    206.0
ALO    206.0
NOR    205.0
SAI    200.0
RUS    175.0
PIA     97.0
STR     74.0
GAS     62.0
OCO     58.0
ALB     27.0
TSU     17.0
BOT     10.0
HUL      9.0
RIC      6.0
ZHO      6.0
MAG      3.0
LAW      2.0
SAR      1.0
DEV      0.0
Name: total_points, dtype: float64


In [26]:
import pandas as pd

# Assuming 'results' is already loaded with appropriate columns: 'driverCode', 'round', 'points'
# Create a pivot table with drivers as rows, rounds as columns, and points as values
results_pivot = results.pivot(index='driverCode', columns='round', values='points')

# Add a 'total_points' column by summing across all races for each driver
results_pivot['total_points'] = results_pivot.sum(axis=1)

# Sort drivers by their total points in descending order
results_pivot = results_pivot.sort_values(by='total_points', ascending=False)

# Drop the 'total_points' column as it was only needed for sorting
results_pivot.drop(columns='total_points', inplace=True)

# Rename columns: Use race names instead of round numbers
# Assuming 'races' is a list where the index corresponds to round numbers (starting from 1)
# Example: races = ['Bahrain GP', 'Saudi Arabia GP', ..., 'Abu Dhabi GP']
race_names = dict(enumerate(races, start=1))  # Map round numbers to race names
results_pivot.rename(columns=race_names, inplace=True)

# Display the final dataframe
print(results_pivot)


KeyError: 'driverCode'

In [ ]:
results.to_csv("Results2024.csv")

In [10]:
temp = ergast.get_race_results(season=2024, round=2)
temp = temp.content[0]
print(temp['constructorId'])


0         red_bull
1         red_bull
2          ferrari
3          mclaren
4     aston_martin
5         mercedes
6          ferrari
7          mclaren
8         mercedes
9             haas
10        williams
11            haas
12          alpine
13        williams
14              rb
15              rb
16          sauber
17          sauber
18    aston_martin
19          alpine
Name: constructorId, dtype: object


In [13]:
from fastf1.ergast import Ergast
import pandas as pd

# Enable cache for FastF1
fastf1.Cache.enable_cache("/Users/ainefernandez/documents/itam/8SEMESTRE/F1 Python")

# Initialize Ergast object
ergast = Ergast()

# Get the race schedule for 2024
races = ergast.get_race_schedule(2024)  # Races in year 2024
results = []

# For each race in the season
for rnd, race in races['raceName'].items():

    # Get results. Note that we use the round no. + 1, because the round no.
    # starts from one (1) instead of zero (0)
    temp = ergast.get_race_results(season=2024, round=rnd + 1)
    temp = temp.content[0]

    try:
        # Check available columns in temp
        print(f"Available columns for race {race}: {temp.columns.tolist()}")

        # If there is a sprint, get the results as well
        sprint = ergast.get_sprint_results(season=2024, round=rnd + 1)
        if sprint.content and sprint.description['round'][0] == rnd + 1:
            sprint_data = sprint.content[0]

            # Merge results with sprint data if available
            temp = pd.merge(temp, sprint_data, on='constructorName', how='left')

            # Add sprint points and race points to get the total
            if 'points_x' in temp.columns and 'points_y' in temp.columns:
                temp['points'] = temp['points_x'] + temp['points_y']
                temp.drop(columns=['points_x', 'points_y'], inplace=True)

        # Add round no. and grand prix name
        temp['round'] = rnd + 1
        temp['race'] = race.removesuffix(' Grand Prix')

        # Ensure that 'driverCode' and other columns exist
        if 'driverCode' in temp.columns and 'constructorName' in temp.columns:
            temp = temp[['round', 'race', 'driverCode', 'constructorName', 'points']]  # Keep useful cols.
            results.append(temp)
        else:
            print(f"Missing 'driverCode' or 'constructorName' in race {race}, skipping.")

    except KeyError as e:
        print(f"KeyError in race {race}: {e}, skipping this race.")

# Append all races into a single dataframe
results_df = pd.concat(results, ignore_index=True)

# Get the unique race names
races = results_df['race'].drop_duplicates()


Available columns for race Bahrain Grand Prix: ['number', 'position', 'positionText', 'points', 'grid', 'laps', 'status', 'driverId', 'driverNumber', 'driverCode', 'driverUrl', 'givenName', 'familyName', 'dateOfBirth', 'driverNationality', 'constructorId', 'constructorUrl', 'constructorName', 'constructorNationality', 'totalRaceTimeMillis', 'totalRaceTime', 'fastestLapRank', 'fastestLapNumber', 'fastestLapTime', 'fastestLapAvgSpeedUnits', 'fastestLapAvgSpeed']
Available columns for race Saudi Arabian Grand Prix: ['number', 'position', 'positionText', 'points', 'grid', 'laps', 'status', 'driverId', 'driverNumber', 'driverCode', 'driverUrl', 'givenName', 'familyName', 'dateOfBirth', 'driverNationality', 'constructorId', 'constructorUrl', 'constructorName', 'constructorNationality', 'totalRaceTimeMillis', 'totalRaceTime', 'fastestLapRank', 'fastestLapNumber', 'fastestLapTime', 'fastestLapAvgSpeedUnits', 'fastestLapAvgSpeed']
Available columns for race Australian Grand Prix: ['number', 'po

Available columns for race United States Grand Prix: ['number', 'position', 'positionText', 'points', 'grid', 'laps', 'status', 'driverId', 'driverNumber', 'driverCode', 'driverUrl', 'givenName', 'familyName', 'dateOfBirth', 'driverNationality', 'constructorId', 'constructorUrl', 'constructorName', 'constructorNationality', 'totalRaceTimeMillis', 'totalRaceTime', 'fastestLapRank', 'fastestLapNumber', 'fastestLapTime', 'fastestLapAvgSpeedUnits', 'fastestLapAvgSpeed']
Missing 'driverCode' or 'constructorName' in race United States Grand Prix, skipping.
Available columns for race Mexico City Grand Prix: ['number', 'position', 'positionText', 'points', 'grid', 'laps', 'status', 'driverId', 'driverNumber', 'driverCode', 'driverUrl', 'givenName', 'familyName', 'dateOfBirth', 'driverNationality', 'constructorId', 'constructorUrl', 'constructorName', 'constructorNationality', 'totalRaceTimeMillis', 'totalRaceTime', 'fastestLapRank', 'fastestLapNumber', 'fastestLapTime', 'fastestLapAvgSpeedUnit

In [22]:
import pandas as pd

# Read the CSV file
resteams = pd.read_csv('Results2024.csv')

# Select only numeric columns (excluding 'Team' column)
numeric_columns = resteams.select_dtypes(include=['number']).columns

# Group by 'Team' and sum the numeric columns (which should be the GP points)
team_points = resteams.groupby(['Team'])[numeric_columns].sum()

# Sum the points for each team across all GPs
team_points['total_points'] = team_points.sum(axis=1)

# Sort the teams by total points in descending order
team_points = team_points.sort_values(by='total_points', ascending=False)

# Drop the 'total_points' column as it was only needed for sorting
team_points.drop(columns='total_points', inplace=True)

# Display the sorted team points
team_points


,Bahrain,Saudi Arabian,Australian,Japanese,Chinese,Miami,Emilia Romagna,Monaco,Canadian,Spanish,...,Dutch,Italian,Azerbaijan,Singapore,United States,Mexico City,Sao Paulo,Las Vegas,Qatar,Abu Dhabi
Team,,,,,,,,,,,,,,,,,,,,,
McLaren,12.0,16.0,27.0,14.0,27.0,28.0,30.0,30.0,28.0,25.0,...,38.0,34.0,38.0,40.0,28.0,22.0,27.0,15.0,32.0,26.0
Ferrari,27.0,16.0,44.0,27.0,31.0,36.0,25.0,40.0,0.0,18.0,...,25.0,37.0,18.0,16.0,55.0,41.0,20.0,27.0,35.0,33.0
Red Bull,44.0,43.0,10.0,44.0,54.0,44.0,29.0,8.0,25.0,29.0,...,26.0,12.0,10.0,19.0,29.0,8.0,32.0,11.0,26.0,8.0
Mercedes,16.0,10.0,0.0,8.0,18.0,12.0,15.0,17.0,28.0,27.0,...,10.0,16.0,17.0,20.0,15.0,22.0,16.0,43.0,21.0,22.0
Aston Martin,3.0,10.0,12.0,8.0,7.0,2.0,2.0,0.0,14.0,0.0,...,1.0,0.0,8.0,4.0,0.0,0.0,0.0,0.0,6.0,2.0
Alpine,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,3.0,3.0,...,2.0,0.0,0.0,0.0,0.0,1.0,35.0,0.0,10.0,6.0
Haas,0.0,7.0,3.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,2.0,7.0,8.0,0.0,4.0,4.0,4.0
RB,0.0,0.0,6.0,1.0,0.0,12.0,1.0,4.0,4.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,8.0,2.0,0.0,0.0
Williams,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,2.0,10.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [23]:
team_points.to_csv("Results2024Teams.csv")

In [4]:
import fastf1
from fastf1.core import Laps
  
import pandas as pd

fastf1.Cache.enable_cache("/Users/ainefernandez/documents/itam/8SEMESTRE/F1 Python")

year = 2024
ses = 'Q'

# Initialize a list to store gap data
gap_data = []

# Loop over all completed Grand Prix sessions (1–24 in this case)
for i in range(1, 25):
    session = fastf1.get_session(year, i, ses)
    try:
        session.load()  # Attempt to load the session
    except DataNotLoadedError as e:
        print(f"Session {i} could not be loaded: {e}")
        continue  # Skip to the next session if loading fails
    except Exception as e:
        print(f"An unexpected error occurred while loading session {i}: {e}")
        continue  # Handle other unexpected errors

    # Get unique drivers
    drivers = pd.unique(session.laps['Driver'])
    fastest_laps_list = []
    
    # Collect the fastest lap for each driver
    for drv in drivers:
        try:
            driver_fastest_lap = session.laps.pick_driver(drv).pick_fastest()
            fastest_laps_list.append(driver_fastest_lap)
        except ValueError:
            # Handle cases where a driver doesn't have a valid lap time
            continue
    
    # Convert the list of fastest laps into a Laps object and sort by lap time
    fastest_laps = Laps(fastest_laps_list).sort_values(by='LapTime').reset_index(drop=True)
    
    # Get the pole position lap (fastest lap overall)
    if not fastest_laps.empty:
        pole_lap = fastest_laps.iloc[0]
        pole_time = pole_lap['LapTime'].total_seconds()  # Pole lap time in seconds
        
        # Calculate gaps to the pole lap and store the data
        for _, row in fastest_laps.iterrows():
            lap_time = row['LapTime'].total_seconds()  # Driver's lap time in seconds
            gap_seconds = lap_time - pole_time  # Gap in seconds
            gap_percentage = (gap_seconds / pole_time) * 100  # Gap as a percentage
            gap_data.append({
                'Driver': row['Driver'],
                'Gap (s)': gap_seconds,
                'Gap (%)': gap_percentage
            })

# Create a DataFrame from the collected gap data
gap_df = pd.DataFrame(gap_data)

# Print the gap data
print(gap_df)

# Group by driver and calculate the average gap in seconds and percentage
avg_gap_to_pole = gap_df.groupby('Driver').mean()
print(avg_gap_to_pole)

# Save the average gap data to a CSV file
# avg_gap_to_pole.to_csv("AvgGapToPole.csv", index=True)


core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v3.0.7]
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '63', '55', '11', '14', '4', '81', '44', '27', '22', '18', '23', '3', '20', '77', '24', '2', '31', '10']
core           INFO 	Loading data for Saudi Arabian Grand Prix - Qualifying [v3.0.7]
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached d

core           INFO 	Loading data for Austrian Grand Prix - Qualifying [v3.0.7]
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '63', '55', '44', '16', '81', '11', '27', '31', '3', '20', '10', '22', '14', '23', '18', '77', '2', '24']
core           INFO 	Loading data for British Grand Prix - Qualifying [v3.0.7]
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data f

DataNotLoadedError: The data you are trying to access has not been loaded yet. See `Session.load`

In [20]:
import fastf1
import pandas as pd

# Initialize a list to store the results for each race
all_race_results = []

# Loop through all the races in the 2024 season (races 1 through 24)
for race_num in range(1, 23):
    try:
        # Load the session for the current race (2024 season, all races)
        session = fastf1.get_session(2023, race_num, 'Q')  # 'Q' for Qualifying session
        session.load()

        # Extract results DataFrame
        results = session.results

        # Get the pole time (time of the driver with Position == 1)
        pole_time = results.loc[results['Position'] == 1, ['Q1', 'Q2', 'Q3']].min(axis=1).dt.total_seconds().iloc[0]

        # Find each driver's fastest time (minimum of Q1, Q2, Q3)
        results['FastestTime'] = results[['Q1', 'Q2', 'Q3']].min(axis=1).dt.total_seconds()

        # Calculate the gap to pole for each driver in seconds
        results['GapInSeconds'] = results['FastestTime'] - pole_time

        # Calculate the gap to pole for each driver in percentage
        results['GapPercentage'] = ((results['FastestTime'] - pole_time) / pole_time) * 100

        # Store the relevant data for each driver
        race_result = results[['Abbreviation', 'FastestTime', 'GapInSeconds', 'GapPercentage']]
        race_result['Race'] = race_num
        all_race_results.append(race_result)

    except Exception as e:
        # If there's an error (e.g., session not available), skip that race
        print(f"Error processing race {race_num}: {e}")

# Combine all the race results into one DataFrame
all_race_results_df = pd.concat(all_race_results, ignore_index=True)

# Calculate the average gap to pole in seconds and percentage by driver across all races
avg_gap_by_driver = all_race_results_df.groupby('Abbreviation').agg(
    AvgGapToPoleSeconds=('GapInSeconds', 'mean'),
    AvgGapToPolePercentage=('GapPercentage', 'mean')
).reset_index()

# Rename the column 'Abbreviation' to 'Driver'
avg_gap_by_driver.rename(columns={'Abbreviation': 'Driver'}, inplace=True)

# Display the results
print(avg_gap_by_driver)

# Optionally, save the results to a CSV file
avg_gap_by_driver.to_csv("AvgGapToPole_ByDriver_2023.csv", index=False)


core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v3.0.7]
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '16', '55', '14', '63', '44', '18', '31', '27', '4', '77', '24', '22', '23', '2', '20', '81', '21', '10']
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_43501/4110387443.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_43501/4110387443.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  race_result['Race'] = race_num
core           INFO 	Loading data for Canadian Grand Prix - Qualifying [v3.0.7]
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req        

req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['55', '1', '16', '63', '11', '23', '81', '44', '4', '14', '22', '40', '27', '77', '2', '24', '10', '31', '20', '18']
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_43501/4110387443.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  race_result['Race'] = race_num
core           INFO 	Loading data for Singapore Grand Prix - Qualifying [v3.0.7]
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_st

req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '55', '1', '63', '10', '23', '2', '77', '20', '14', '44', '11', '27', '18', '3', '4', '31', '24', '81', '22']
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_43501/4110387443.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

   Driver  AvgGapToPoleSeconds  AvgGapToPolePercentage
0     ALB             1.796727                2.008633
1     ALO             0.795091                0.936336
2     BOT             2.293045                2.781882
3     DEV             2.141333                2.664135
4     GAS             2.017000                2.410058
5     HAM             1.035818                1.319728
6     HUL             2.392273                2.757483
7     LAW             3.980800                5.297652
8     LEC             0.461136                0.586548
9     MAG             2.653545                3.229331
10    NOR             0.971409                1.154324
11    OCO             2.054409                2.464336
12    PER             0.906381                1.118393
13    PIA             1.028318                1.227886
14    RIC             3.158714                3.193203
15    RUS             0.759273                0.920198
16    SAI             0.562955                0.677911
17    SAR 

/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_43501/4110387443.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  race_result['Race'] = race_num


In [13]:
session = fastf1.get_session(2024, "Abu Dhabi", "Q")
session.load()

# Extract results DataFrame
results = session.results

results

core           INFO 	Loading data for Abu Dhabi Grand Prix - Qualifying [v3.0.7]
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['4', '81', '55', '27', '1', '10', '63', '14', '77', '11', '22', '30', '18', '16', '20', '23', '24', '44', '43', '61']


,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,...,CountryCode,Position,ClassifiedPosition,GridPosition,Q1,Q2,Q3,Time,Status,Points
4,4,L NORRIS,NOR,norris,McLaren,FF8000,mclaren,Lando,Norris,Lando Norris,...,GBR,1.0,,NaN,0 days 00:01:23.682000,0 days 00:01:23.098000,0 days 00:01:22.595000,NaT,,NaN
81,81,O PIASTRI,PIA,piastri,McLaren,FF8000,mclaren,Oscar,Piastri,Oscar Piastri,...,AUS,2.0,,NaN,0 days 00:01:23.640000,0 days 00:01:23.199000,0 days 00:01:22.804000,NaT,,NaN
55,55,C SAINZ,SAI,sainz,Ferrari,E80020,ferrari,Carlos,Sainz,Carlos Sainz,...,ESP,3.0,,NaN,0 days 00:01:23.487000,0 days 00:01:22.985000,0 days 00:01:22.824000,NaT,,NaN
27,27,N HULKENBERG,HUL,hulkenberg,Haas F1 Team,B6BABD,haas,Nico,Hulkenberg,Nico Hulkenberg,...,GER,4.0,,NaN,0 days 00:01:23.722000,0 days 00:01:23.040000,0 days 00:01:22.886000,NaT,,NaN
1,1,M VERSTAPPEN,VER,max_verstappen,Red Bull Racing,3671C6,red_bull,Max,Verstappen,Max Verstappen,...,NED,5.0,,NaN,0 days 00:01:23.516000,0 days 00:01:22.998000,0 days 00:01:22.945000,NaT,,NaN
10,10,P GASLY,GAS,gasly,Alpine,0093cc,alpine,Pierre,Gasly,Pierre Gasly,...,FRA,6.0,,NaN,0 days 00:01:23.548000,0 days 00:01:23.086000,0 days 00:01:22.984000,NaT,,NaN
63,63,G RUSSELL,RUS,russell,Mercedes,27F4D2,mercedes,George,Russell,George Russell,...,GBR,7.0,,NaN,0 days 00:01:23.678000,0 days 00:01:23.283000,0 days 00:01:23.132000,NaT,,NaN
14,14,F ALONSO,ALO,alonso,Aston Martin,229971,aston_martin,Fernando,Alonso,Fernando Alonso,...,ESP,8.0,,NaN,0 days 00:01:23.794000,0 days 00:01:23.268000,0 days 00:01:23.196000,NaT,,NaN
77,77,V BOTTAS,BOT,bottas,Kick Sauber,52e252,sauber,Valtteri,Bottas,Valtteri Bottas,...,FIN,9.0,,NaN,0 days 00:01:23.481000,0 days 00:01:23.341000,0 days 00:01:23.204000,NaT,,NaN
11,11,S PEREZ,PER,perez,Red Bull Racing,3671C6,red_bull,Sergio,Perez,Sergio Perez,...,MEX,10.0,,NaN,0 days 00:01:23.559000,0 days 00:01:23.379000,0 days 00:01:23.264000,NaT,,NaN


In [29]:
import fastf1
import pandas as pd

# Function to calculate the relative percentage gap between teammates
def calculate_relative_percentage_gap(time1, time2):
    return abs(time1 - time2) / time1 * 100  # relative percentage gap

# Initialize an empty DataFrame to store the results
season_percentage_gaps = pd.DataFrame(columns=['Race', 'Team', 'Driver1', 'Driver2', 'PercentageGap'])

# Loop through each race in the 2024 season
for race_id in range(1, 25):  # Loop through all races in the season
    race = fastf1.get_session(2024, race_id, 'Q')  # Get qualifying session
    race.load()

    # Extract the results DataFrame
    results = race.results

    # Ensure both drivers are present in the session
    if results['Position'].notna().sum() >= 2:
        # Get the fastest lap time for both drivers (min of Q1, Q2, Q3)
        results['FastestTime'] = results[['Q1', 'Q2', 'Q3']].min(axis=1).dt.total_seconds()
        
        # Compare teammates, assuming they have the same TeamName
        for team in results['TeamName'].unique():
            team_results = results[results['TeamName'] == team]
            
            if len(team_results) == 2:  # Ensure both drivers are in the results
                driver1 = team_results.iloc[0]
                driver2 = team_results.iloc[1]

                # Calculate the relative percentage gap between teammates
                percentage_gap = calculate_relative_percentage_gap(driver1['FastestTime'], driver2['FastestTime'])

                # Create a DataFrame to store the current race's data
                temp_df = pd.DataFrame([{
                    'Race': race.event['EventName'],
                    'Team': team,
                    'Driver1': driver1['Abbreviation'],
                    'Driver2': driver2['Abbreviation'],
                    'PercentageGap': percentage_gap
                }])

                # Concatenate the temp_df to the main DataFrame
                season_percentage_gaps = pd.concat([season_percentage_gaps, temp_df], ignore_index=True)

# Calculate average percentage gap for the whole season for each driver pair
average_percentage_gap = season_percentage_gaps.groupby(['Team', 'Driver1', 'Driver2'])['PercentageGap'].mean().reset_index()

# Now, calculate the final gap for each team by comparing the average gap for both drivers
team_results_final = average_percentage_gap.groupby('Team').apply(
    lambda group: group.loc[group['PercentageGap'].idxmin()]  # Get the driver pair with the smallest average gap
).reset_index(drop=True)

# Add a column to indicate the faster driver for each team
team_results_final['FasterDriver'] = team_results_final.apply(
    lambda row: row['Driver1'] if row['PercentageGap'] < row['PercentageGap'] else row['Driver2'],
    axis=1
)

# Display the results: Team, Percentage Gap, Faster Driver
final_results = team_results_final[['Team', 'PercentageGap', 'FasterDriver']]
print(final_results)


core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v3.0.7]
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '63', '55', '11', '14', '4', '81', '44', '27', '22', '18', '23', '3', '20', '77', '24', '2', '31', '10']
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_43501/1898941927.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is depre

req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['4', '1', '44', '63', '16', '55', '10', '11', '31', '81', '14', '77', '27', '18', '24', '20', '22', '3', '23', '2']
core           INFO 	Loading data for Austrian Grand Prix - Qualifying [v3.0.7]
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            I

req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['55', '1', '4', '16', '63', '44', '20', '10', '23', '27', '22', '30', '14', '18', '77', '43', '81', '11', '31', '24']
core           INFO 	Loading data for São Paulo Grand Prix - Qualifying [v3.0.7]
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
core        WARNING 	Failed to load extended driver information!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	

               Team  PercentageGap FasterDriver
0            Alpine       0.741646          OCO
1    Alpine F1 Team       0.601096          OCO
2      Aston Martin       0.479318          ALO
3           Ferrari       0.333796          LEC
4      Haas F1 Team       0.280523          HUL
5       Kick Sauber       0.837575          ZHO
6           McLaren       0.401219          PIA
7          Mercedes       0.663938          RUS
8                RB       0.266898          LAW
9        RB F1 Team       0.790371          LAW
10         Red Bull       0.952165          PER
11  Red Bull Racing       0.206260          VER
12           Sauber       0.923412          BOT
13         Williams       0.541220          ALB


/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_43501/1898941927.py:51: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  team_results_final = average_percentage_gap.groupby('Team').apply(
